In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!ls

drive  sample_data  spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

spark-2.4.4-bin-hadoop2.7/
spark-2.4.4-bin-hadoop2.7/R/
spark-2.4.4-bin-hadoop2.7/R/lib/
spark-2.4.4-bin-hadoop2.7/R/lib/sparkr.zip
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/R.css
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/html/00Index.html
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/aliases.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/AnIndex
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdx
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/SparkR.rdb
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/help/paths.rds
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-2.4.4-bin-hadoop2.7/R/lib/SparkR/tests/testthat/te

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#loading the dataset 

df = spark.read.csv('/content/drive/My Drive/tick_data.csv',inferSchema=True, header =True)

In [0]:
df.head(5)

[Row(DATE=20120103, TIME_S='09:04:08', SYM_ROOT='AAPL', SIZE=500, TRADE=409.21),
 Row(DATE=20120103, TIME_S='09:06:49', SYM_ROOT='AAPL', SIZE=100, TRADE=409.41),
 Row(DATE=20120103, TIME_S='09:21:10', SYM_ROOT='AAPL', SIZE=100, TRADE=409.49),
 Row(DATE=20120103, TIME_S='09:26:28', SYM_ROOT='AAPL', SIZE=100, TRADE=409.04),
 Row(DATE=20120103, TIME_S='09:29:55', SYM_ROOT='AAPL', SIZE=100, TRADE=410.3)]

In [0]:
df.printSchema()

root
 |-- DATE: integer (nullable = true)
 |-- TIME_S: string (nullable = true)
 |-- SYM_ROOT: string (nullable = true)
 |-- SIZE: integer (nullable = true)
 |-- TRADE: double (nullable = true)



In [0]:
df.registerTempTable("tick_table")

Answer 1 


In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import*

In [0]:
data = spark.sql('SELECT SYM_ROOT,DATE, hour(TIME_S) as TIME_H,sum(size) as SIZE_H from tick_table group by sym_root, hour(TIME_S),date order by sym_root,date,hour(TIME_S)')

In [0]:
data.show()

+--------+--------+------+------+
|SYM_ROOT|    DATE|TIME_H|SIZE_H|
+--------+--------+------+------+
|    AAPL|20120103|     9| 21092|
|    AAPL|20120103|    10| 12041|
|    AAPL|20120103|    11|  9473|
|    AAPL|20120103|    12|  5718|
|    AAPL|20120103|    13|  8103|
|    AAPL|20120103|    14|  7783|
|    AAPL|20120103|    15| 15334|
|    GOOG|20120103|     9|  8314|
|    GOOG|20120103|    10|  8080|
|    GOOG|20120103|    11|  6776|
|    GOOG|20120103|    12|  6619|
|    GOOG|20120103|    13|  6517|
|    GOOG|20120103|    14|  8453|
|    GOOG|20120103|    15| 17496|
|     IBM|20120103|     9| 14064|
|     IBM|20120103|    10| 12055|
|     IBM|20120103|    11| 12863|
|     IBM|20120103|    12| 11694|
|     IBM|20120103|    13|  5109|
|     IBM|20120103|    14|  9033|
+--------+--------+------+------+
only showing top 20 rows



Answer 2

In [0]:
part2_data = spark.sql('SELECT SYM_ROOT,DATE, hour(TIME_S) as TIME_H,sum(size) as SIZE_H,(last(trade)-first(trade))/first(trade) as RETURN_H from tick_table group by sym_root, hour(TIME_S),date')

In [0]:
part2_data.show()

+--------+--------+------+------+--------------------+
|SYM_ROOT|    DATE|TIME_H|SIZE_H|            RETURN_H|
+--------+--------+------+------+--------------------+
|    GOOG|20120103|    12|  6619|-0.00377609280125...|
|    AAPL|20120103|    11|  9473|-1.45857642940495...|
|    MSFT|20120103|    14| 13715|0.001856665428889...|
|    MSFT|20120103|    10| 23579|0.007906626506024129|
|    GOOG|20120103|    11|  6776|0.003213387091884...|
|    GOOG|20120103|    15| 17496|9.169071669071874E-4|
|    MSFT|20120103|    13| 12632|0.001115656377835...|
|     IBM|20120103|     9| 14064|0.003090531251665224|
|     IBM|20120103|    14|  9033|8.028689182678223E-4|
|    GOOG|20120103|    10|  8080|8.037366170272154E-4|
|     IBM|20120103|    11| 12863|-0.00201548742972...|
|    AAPL|20120103|    10| 12041|-0.00109308200544...|
|    GOOG|20120103|     9|  8314|0.006056815061178277|
|     IBM|20120103|    10| 12055|7.964742738808329E-4|
|    MSFT|20120103|    12| 18342|-3.71885459278600...|
|    AAPL|

In [0]:
from pyspark.sql.functions import col

Answer 3

In [0]:
cols = ["SYM_ROOT", "DATE", "TIME_H"]

part3_data = part2_data.orderBy(cols, ascending=True)

In [0]:
part3_data.show()

+--------+--------+------+------+--------------------+
|SYM_ROOT|    DATE|TIME_H|SIZE_H|            RETURN_H|
+--------+--------+------+------+--------------------+
|    AAPL|20120103|     9| 21092|0.005302900711126356|
|    AAPL|20120103|    10| 12041|-0.00109308200544...|
|    AAPL|20120103|    11|  9473|-1.45857642940495...|
|    AAPL|20120103|    12|  5718|-0.00250474198725...|
|    AAPL|20120103|    13|  8103|0.002315604738458...|
|    AAPL|20120103|    14|  7783|0.002066567795580...|
|    AAPL|20120103|    15| 15334|-0.00172259019336...|
|    GOOG|20120103|     9|  8314|0.006056815061178277|
|    GOOG|20120103|    10|  8080|8.037366170272154E-4|
|    GOOG|20120103|    11|  6776|0.003213387091884...|
|    GOOG|20120103|    12|  6619|-0.00377609280125...|
|    GOOG|20120103|    13|  6517|1.363305865245271...|
|    GOOG|20120103|    14|  8453|0.006965897389302839|
|    GOOG|20120103|    15| 17496|9.169071669071874E-4|
|     IBM|20120103|     9| 14064|0.003090531251665224|
|     IBM|

In [0]:
from pyspark import SparkContext 
from pyspark import HiveContext
from pyspark.sql.functions import*
import pyspark.sql.functions as fn 
sc = SparkContext.getOrCreate()
sqlContext=HiveContext(sc)

df = sqlContext.read.csv('/content/drive/My Drive/tick_data.csv')

# Displays the content of the DataFrame to stdout
df.show()





+--------+--------+--------+----+------+
|     _c0|     _c1|     _c2| _c3|   _c4|
+--------+--------+--------+----+------+
|    DATE|  TIME_S|SYM_ROOT|SIZE| TRADE|
|20120103|09:04:08|    AAPL| 500|409.21|
|20120103|09:06:49|    AAPL| 100|409.41|
|20120103|09:21:10|    AAPL| 100|409.49|
|20120103|09:26:28|    AAPL| 100|409.04|
|20120103|09:29:55|    AAPL| 100| 410.3|
|20120103|09:30:00|    AAPL| 200|409.91|
|20120103|09:30:04|    AAPL| 100|410.11|
|20120103|09:30:04|    AAPL| 200|410.05|
|20120103|09:30:06|    AAPL| 100| 410.2|
|20120103|09:30:20|    AAPL| 140| 409.9|
|20120103|09:30:39|    AAPL| 100|409.82|
|20120103|09:30:49|    AAPL| 100|409.76|
|20120103|09:30:50|    AAPL| 100| 409.8|
|20120103|09:30:50|    AAPL| 200| 409.8|
|20120103|09:31:04|    AAPL| 150|409.53|
|20120103|09:31:25|    AAPL| 174|409.79|
|20120103|09:31:50|    AAPL| 100|409.48|
|20120103|09:31:55|    AAPL| 200|409.44|
|20120103|09:32:29|    AAPL| 100|409.95|
+--------+--------+--------+----+------+
only showing top

In [0]:
part3_data.toPandas().to_csv('/content/drive/My Drive/tick1_data.csv', index=None, header=True)